<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/SPARK_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

import findspark
findspark.init()

In [2]:
file_download_link = "https://drive.google.com/u/0/uc?id=1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi&export=download" 
!wget -O paper_graph_adj_list.txt --no-check-certificate "$file_download_link"

file_download_link = "https://drive.google.com/u/0/uc?id=1qmwKYipNwJZyheNfVxW8GwzhbY6J2nPU&export=download" 
!wget -O paper_graph_node_list.txt --no-check-certificate "$file_download_link"

--2021-04-25 18:16:37--  https://drive.google.com/u/0/uc?id=1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi&export=download
Resolving drive.google.com (drive.google.com)... 172.217.193.100, 172.217.193.113, 172.217.193.102, ...
Connecting to drive.google.com (drive.google.com)|172.217.193.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ndnrfl23fd0rqnna5nns3h2s2mm0qmg4/1619374575000/14577798666193718332/*/1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi?e=download [following]
--2021-04-25 18:16:38--  https://doc-0s-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ndnrfl23fd0rqnna5nns3h2s2mm0qmg4/1619374575000/14577798666193718332/*/1BgY3iHSZ0ImXc6atr61YQRcloxDwNrvi?e=download
Resolving doc-0s-70-docs.googleusercontent.com (doc-0s-70-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-0s-70-docs.googleusercontent.com

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [4]:
def create_all_links(all_data):

  result = []
  cnt = len(all_data)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      if (all_data[i] < all_data[j]):
        result.append( ((all_data[i], all_data[j]), 0))
      else:
        result.append( ((all_data[j], all_data[i]), 0))

  return result

def calc_cc(key, value):

  result = []
  cnt = len(value)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      if (value[i] < value[j]):
        result.append( ((value[i], value[j]), key))
      else:
        result.append( ((value[j], value[i]), key))

  return result

def calc_car(key, value):

  result = []

  cnt = len(value)

  for i in range(cnt):
    for j in range(i + 1, cnt):
      for k in range(j +1, cnt):
        
        if (key < value[k]):
          first_data = (key, value[k])
        else:
          first_data = (value[k], key)

        if (value[i] < value[j]):
          second_data = (value[i], value[j])
        else:
          second_data = (value[j], value[i])
    
        result.append( (second_data , (first_data, 1)))

        if (key < value[j]):
          first_data = (key, value[j])
        else:
          first_data = (value[j], key)

        if (value[i] < value[k]):
          second_data = (value[i], value[k])
        else:
          second_data = (value[k], value[i])
    
        result.append( (second_data , (first_data, 1)))
        
        if (key < value[i]):
          first_data = (key, value[i])
        else:
          first_data = (value[i], key)

        if (value[k] < value[j]):
          second_data = (value[k], value[j])
        else:
          second_data = (value[j], value[k])
    
        result.append( (second_data , (first_data, 1)))
  return result

def Convert(lst):
  result = {}

  for item in lst:
    if (item[0] in result):
      result[item[0]] = result[item[0]] + 1
    else:
      result[item[0]] = 1

  return result

def remove_only_one_time_appers(dic):
  return sum(y / 2 for x,y in dic.items() if y> 1)

def car_grouped_mix(key, value):
  aggrigated = remove_only_one_time_appers(Convert(value))
  return (key, aggrigated)

In [5]:
df = spark.sparkContext.textFile("paper_graph_adj_list.txt")
nodes = spark.sparkContext.textFile("paper_graph_node_list.txt")

temp_var = df.map(lambda k: k.split(" "))
temp_nodes = nodes.map(lambda k: k.split(" "))

all_links = temp_nodes.flatMap(lambda k : create_all_links(k))
data = temp_var.map(lambda k : (k[0], k[1::]))

cc_reduce = data.flatMap(lambda x: calc_cc(x[0], x[1]))
data_cn = cc_reduce.groupByKey().map(lambda x : (x[0], len(list(x[1]))))

car_reduce = data.flatMap(lambda x: calc_car(x[0], x[1]))
grouped = car_reduce.groupByKey().map(lambda x : (x[0], list(x[1])))

car_grouped_reduce = grouped.map(lambda x: (car_grouped_mix(x[0], x[1])))

joined_data = car_grouped_reduce.union(all_links)
join_by_all_links = joined_data.reduceByKey(lambda x, y: x + y)

car_for_all_links = join_by_all_links.union(data_cn).reduceByKey(lambda x, y: x * y)
car_for_all_links.collect()

[(('A', 'B'), 2.0),
 (('A', 'F'), 0),
 (('D', 'G'), 0),
 (('G', 'I'), 0),
 (('D', 'F'), 2.0),
 (('B', 'D'), 0),
 (('F', 'G'), 0),
 (('B', 'E'), 0),
 (('G', 'H'), 0),
 (('E', 'F'), 0),
 (('E', 'H'), 0),
 (('C', 'D'), 0),
 (('D', 'J'), 0),
 (('D', 'K'), 0),
 (('C', 'G'), 0),
 (('C', 'I'), 0),
 (('G', 'K'), 0),
 (('C', 'H'), 0),
 (('H', 'J'), 0),
 (('H', 'K'), 0),
 (('D', 'E'), 0),
 (('B', 'H'), 0),
 (('A', 'D'), 0),
 (('B', 'G'), 0),
 (('H', 'I'), 0),
 (('D', 'H'), 0),
 (('F', 'I'), 0),
 (('A', 'C'), 0),
 (('B', 'C'), 0),
 (('E', 'J'), 0),
 (('I', 'J'), 0),
 (('B', 'J'), 0),
 (('B', 'K'), 0),
 (('A', 'K'), 0),
 (('F', 'K'), 0),
 (('B', 'F'), 0),
 (('A', 'H'), 0),
 (('C', 'J'), 0),
 (('C', 'K'), 0),
 (('D', 'I'), 0),
 (('E', 'G'), 0),
 (('E', 'I'), 0),
 (('F', 'H'), 0),
 (('A', 'G'), 2.0),
 (('A', 'E'), 0),
 (('J', 'K'), 0),
 (('A', 'I'), 0),
 (('B', 'I'), 0),
 (('C', 'E'), 2.0),
 (('E', 'K'), 0),
 (('G', 'J'), 0),
 (('I', 'K'), 0),
 (('C', 'F'), 0),
 (('A', 'J'), 0),
 (('F', 'J'), 0)]

In [6]:
data_cn_for_all_links = data_cn.union(all_links).reduceByKey(lambda x, y: x + y)
data_cn_for_all_links.collect()

[(('D', 'E'), 1),
 (('A', 'B'), 2),
 (('A', 'F'), 1),
 (('B', 'F'), 1),
 (('D', 'G'), 1),
 (('A', 'H'), 1),
 (('G', 'I'), 1),
 (('F', 'H'), 1),
 (('B', 'H'), 0),
 (('C', 'J'), 0),
 (('C', 'K'), 0),
 (('D', 'I'), 0),
 (('E', 'G'), 0),
 (('E', 'I'), 0),
 (('C', 'D'), 2),
 (('C', 'E'), 2),
 (('A', 'C'), 2),
 (('B', 'C'), 1),
 (('D', 'J'), 1),
 (('D', 'K'), 1),
 (('C', 'G'), 1),
 (('C', 'I'), 1),
 (('E', 'K'), 1),
 (('E', 'J'), 1),
 (('G', 'J'), 0),
 (('G', 'K'), 0),
 (('I', 'J'), 0),
 (('I', 'K'), 0),
 (('D', 'F'), 2),
 (('A', 'D'), 1),
 (('A', 'G'), 2),
 (('B', 'D'), 2),
 (('B', 'G'), 1),
 (('F', 'G'), 1),
 (('A', 'E'), 1),
 (('B', 'E'), 1),
 (('J', 'K'), 1),
 (('A', 'I'), 1),
 (('G', 'H'), 1),
 (('H', 'I'), 1),
 (('F', 'I'), 1),
 (('B', 'I'), 0),
 (('D', 'H'), 0),
 (('E', 'F'), 0),
 (('E', 'H'), 0),
 (('C', 'F'), 2),
 (('B', 'J'), 1),
 (('B', 'K'), 1),
 (('C', 'H'), 1),
 (('A', 'J'), 0),
 (('A', 'K'), 0),
 (('F', 'J'), 0),
 (('F', 'K'), 0),
 (('H', 'J'), 0),
 (('H', 'K'), 0)]

In [60]:
degree_df = data.map(lambda x: (x[0], len(x[1])))
degree_cross_joined = degree_df.cartesian(degree_df)
pa = degree_cross_joined.map(lambda x: ((x[0][0], x[1][0]), x[0][1] * x[1][1])).filter(lambda x: x[0][1] != x[0][0])
pa.collect()

[(('A', 'B'), 9),
 (('A', 'C'), 15),
 (('A', 'D'), 12),
 (('A', 'E'), 12),
 (('B', 'A'), 9),
 (('C', 'A'), 15),
 (('B', 'C'), 15),
 (('C', 'B'), 15),
 (('B', 'D'), 12),
 (('B', 'E'), 12),
 (('C', 'D'), 20),
 (('C', 'E'), 20),
 (('D', 'A'), 12),
 (('E', 'A'), 12),
 (('D', 'B'), 12),
 (('D', 'C'), 20),
 (('E', 'B'), 12),
 (('E', 'C'), 20),
 (('D', 'E'), 16),
 (('E', 'D'), 16),
 (('A', 'F'), 15),
 (('A', 'G'), 6),
 (('A', 'H'), 6),
 (('A', 'I'), 6),
 (('A', 'J'), 6),
 (('A', 'K'), 6),
 (('B', 'F'), 15),
 (('C', 'F'), 25),
 (('B', 'G'), 6),
 (('B', 'H'), 6),
 (('C', 'G'), 10),
 (('C', 'H'), 10),
 (('B', 'I'), 6),
 (('B', 'J'), 6),
 (('B', 'K'), 6),
 (('C', 'I'), 10),
 (('C', 'J'), 10),
 (('C', 'K'), 10),
 (('D', 'F'), 20),
 (('E', 'F'), 20),
 (('D', 'G'), 8),
 (('D', 'H'), 8),
 (('E', 'G'), 8),
 (('E', 'H'), 8),
 (('D', 'I'), 8),
 (('D', 'J'), 8),
 (('D', 'K'), 8),
 (('E', 'I'), 8),
 (('E', 'J'), 8),
 (('E', 'K'), 8),
 (('F', 'A'), 15),
 (('F', 'B'), 15),
 (('F', 'C'), 25),
 (('F', 'D'), 2

In [86]:
cc_reverse = cc_reduce.map(lambda x: (x[1], x[0]))
ra = degree_df.join(cc_reverse).map(lambda x : (x[1][1], 1 / x[1][0])).reduceByKey(lambda a, b: a + b)
ra_all_links = ra.union(all_links).reduceByKey(lambda x, y: x + y)
ra_all_links.collect()

[(('G', 'I'), 0.2),
 (('A', 'B'), 0.45),
 (('A', 'F'), 0.2),
 (('D', 'G'), 0.2),
 (('D', 'F'), 0.5333333333333333),
 (('G', 'H'), 0.2),
 (('B', 'D'), 0.45),
 (('F', 'G'), 0.2),
 (('B', 'E'), 0.25),
 (('E', 'F'), 0),
 (('E', 'H'), 0),
 (('C', 'D'), 0.6666666666666666),
 (('C', 'G'), 0.2),
 (('C', 'I'), 0.2),
 (('D', 'J'), 0.25),
 (('D', 'K'), 0.25),
 (('C', 'H'), 0.2),
 (('G', 'K'), 0),
 (('H', 'J'), 0),
 (('H', 'K'), 0),
 (('D', 'E'), 0.3333333333333333),
 (('H', 'I'), 0.2),
 (('F', 'I'), 0.5),
 (('A', 'D'), 0.2),
 (('B', 'G'), 0.2),
 (('B', 'H'), 0),
 (('D', 'H'), 0),
 (('E', 'J'), 0.5),
 (('A', 'C'), 0.45),
 (('B', 'C'), 0.25),
 (('B', 'J'), 0.25),
 (('B', 'K'), 0.25),
 (('A', 'K'), 0),
 (('F', 'K'), 0),
 (('I', 'J'), 0),
 (('A', 'H'), 0.2),
 (('B', 'F'), 0.2),
 (('F', 'H'), 0.5),
 (('A', 'G'), 0.4),
 (('A', 'I'), 0.2),
 (('A', 'E'), 0.25),
 (('J', 'K'), 0.25),
 (('B', 'I'), 0),
 (('C', 'J'), 0),
 (('C', 'K'), 0),
 (('D', 'I'), 0),
 (('E', 'G'), 0),
 (('E', 'I'), 0),
 (('E', 'K'), 0.

In [88]:
import numpy as np
adamicadar = degree_df.join(cc_reverse).map(lambda x : (x[1][1], 1 / np.log10(x[1][0]))).reduceByKey(lambda a, b: a + b)
adamicadar_all_links = adamicadar.union(all_links).reduceByKey(lambda x, y: x + y)
adamicadar_all_links.collect()

[(('G', 'I'), 1.430676558073393),
 (('A', 'B'), 3.0916406055170738),
 (('A', 'F'), 1.430676558073393),
 (('D', 'G'), 1.430676558073393),
 (('D', 'F'), 3.5265798323627777),
 (('G', 'H'), 1.430676558073393),
 (('B', 'D'), 3.0916406055170738),
 (('F', 'G'), 1.430676558073393),
 (('B', 'E'), 1.660964047443681),
 (('E', 'F'), 0),
 (('E', 'H'), 0),
 (('C', 'D'), 4.19180654857877),
 (('C', 'G'), 1.430676558073393),
 (('C', 'I'), 1.430676558073393),
 (('D', 'J'), 1.660964047443681),
 (('D', 'K'), 1.660964047443681),
 (('C', 'H'), 1.430676558073393),
 (('G', 'K'), 0),
 (('H', 'J'), 0),
 (('H', 'K'), 0),
 (('D', 'E'), 2.095903274289385),
 (('H', 'I'), 1.430676558073393),
 (('F', 'I'), 3.321928094887362),
 (('A', 'D'), 1.430676558073393),
 (('B', 'G'), 1.430676558073393),
 (('B', 'H'), 0),
 (('D', 'H'), 0),
 (('E', 'J'), 3.321928094887362),
 (('A', 'C'), 3.0916406055170738),
 (('B', 'C'), 1.660964047443681),
 (('B', 'J'), 1.660964047443681),
 (('B', 'K'), 1.660964047443681),
 (('A', 'K'), 0),
 ((

In [110]:
from collections import Counter
data.cartesian(data).map(lambda x: ((x[0][0], x[1][0]) , x[0][1] + x[1][1])).filter(lambda x: x[0][1] != x[0][0]).map(lambda x: (x[0], len(Counter(x[1]).keys()))).collect()

[(('A', 'B'), 4),
 (('A', 'C'), 6),
 (('A', 'D'), 6),
 (('A', 'E'), 6),
 (('B', 'A'), 4),
 (('C', 'A'), 6),
 (('B', 'C'), 7),
 (('C', 'B'), 7),
 (('B', 'D'), 5),
 (('B', 'E'), 6),
 (('C', 'D'), 7),
 (('C', 'E'), 7),
 (('D', 'A'), 6),
 (('E', 'A'), 6),
 (('D', 'B'), 5),
 (('D', 'C'), 7),
 (('E', 'B'), 6),
 (('E', 'C'), 7),
 (('D', 'E'), 7),
 (('E', 'D'), 7),
 (('A', 'F'), 7),
 (('A', 'G'), 3),
 (('A', 'H'), 4),
 (('A', 'I'), 4),
 (('A', 'J'), 5),
 (('A', 'K'), 5),
 (('B', 'F'), 7),
 (('C', 'F'), 8),
 (('B', 'G'), 4),
 (('B', 'H'), 5),
 (('C', 'G'), 6),
 (('C', 'H'), 6),
 (('B', 'I'), 5),
 (('B', 'J'), 4),
 (('B', 'K'), 4),
 (('C', 'I'), 6),
 (('C', 'J'), 7),
 (('C', 'K'), 7),
 (('D', 'F'), 7),
 (('E', 'F'), 9),
 (('D', 'G'), 5),
 (('D', 'H'), 6),
 (('E', 'G'), 6),
 (('E', 'H'), 6),
 (('D', 'I'), 6),
 (('D', 'J'), 5),
 (('D', 'K'), 5),
 (('E', 'I'), 6),
 (('E', 'J'), 5),
 (('E', 'K'), 5),
 (('F', 'A'), 7),
 (('F', 'B'), 7),
 (('F', 'C'), 8),
 (('F', 'D'), 7),
 (('F', 'E'), 9),
 (('G', 'A

In [111]:
from collections import Counter
common_cc = cc_reduce.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda a,b: a+b).map(lambda x: (x[0], len(x[1])))
union_cc = data.cartesian(data).map(lambda x: ((x[0][0], x[1][0]) , x[0][1] + x[1][1])).filter(lambda x: x[0][1] != x[0][0]).map(lambda x: (x[0], len(Counter(x[1]).keys())))
jaccard = common_cc.join(union_cc).map(lambda x: (x[0], x[1][0] / x[1][1]))
jaccard_all_links = jaccard.union(all_links).reduceByKey(lambda x, y: x + y)
jaccard_all_links.collect()

[(('A', 'B'), 0.5),
 (('G', 'I'), 0.3333333333333333),
 (('B', 'F'), 0.14285714285714285),
 (('A', 'H'), 0.25),
 (('F', 'H'), 0.16666666666666666),
 (('C', 'J'), 0),
 (('D', 'I'), 0),
 (('E', 'I'), 0),
 (('C', 'D'), 0.2857142857142857),
 (('D', 'K'), 0.2),
 (('C', 'G'), 0.16666666666666666),
 (('B', 'C'), 0.14285714285714285),
 (('C', 'E'), 0.2857142857142857),
 (('E', 'K'), 0.2),
 (('G', 'K'), 0),
 (('I', 'J'), 0),
 (('D', 'F'), 0.2857142857142857),
 (('F', 'G'), 0.16666666666666666),
 (('A', 'D'), 0.16666666666666666),
 (('H', 'I'), 0.3333333333333333),
 (('A', 'G'), 0.6666666666666666),
 (('A', 'E'), 0.16666666666666666),
 (('B', 'I'), 0),
 (('E', 'F'), 0),
 (('B', 'J'), 0.25),
 (('C', 'F'), 0.25),
 (('A', 'K'), 0),
 (('F', 'K'), 0),
 (('H', 'J'), 0),
 (('A', 'F'), 0.14285714285714285),
 (('D', 'G'), 0.2),
 (('D', 'E'), 0.14285714285714285),
 (('B', 'H'), 0),
 (('C', 'K'), 0),
 (('E', 'G'), 0),
 (('D', 'J'), 0.2),
 (('C', 'I'), 0.16666666666666666),
 (('A', 'C'), 0.3333333333333333)